In [8]:
!pip install tiledb
print("")

from datetime import datetime
import firecloud.api as FAPI
import json
import numpy as np
import os
import pandas as pd
import pprint
import tiledb
# from google.cloud import storage

# # If you don't specify credentials when constructing the client, the
# # client library will look for credentials in the environment.
# storage_client = storage.Client()

# # Make an authenticated API request
# buckets = list(storage_client.list_buckets())
# print(buckets)
# print(len(buckets))


pp = pprint.PrettyPrinter(indent=4)

BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
CATALOG_WORKSPACE_NAMESPACE = "anvil-datastorage"

# print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

print("")

# Print TileDB-Py version
print(tiledb.__version__)

# Print TileDB core version
print(tiledb.libtiledb.version())


print("I am ", FAPI.whoami())
print("Workspace Bucke: ", WORKSPACE_BUCKET)

## Read in tags list
workspacees = pd.read_csv('AnVILCatalogWorkspacesPublicMetadata-2022-02-19.csv')
# workspaceTags = workspaceTags[["name","sheet:consortium", "tagsSheet:tag:tags:dbGaP"]]
workspacees.set_index("name", inplace=True)
len(workspacees)




0.12.2
(2, 6, 2)
I am  pet-117272931645288568532@terra-a3b95d52.iam.gserviceaccount.com
Workspace Bucke:  gs://fc-6f27926c-489e-469f-adc5-bb2d9cfe7eec


516

In [17]:
#https://gist.github.com/ihnorton/1544bb169b4ec8cd385e9c3e756c2bd5

array_url = WORKSPACE_BUCKET+"/tiledb/dscatalog"
array_url = array_url.replace("gs","gcs")
print(array_url)

dataTypeDim  = tiledb.Dim(name="dataType", tile=None, dtype="ascii")
indicationDim  = tiledb.Dim(name="indication", tile=None, dtype="ascii")
dom = tiledb.Domain(indicationDim, dataTypeDim)
attr = tiledb.Attr(name="workspace", dtype=np.str_)
schema = tiledb.ArraySchema(domain=dom, sparse=True, attrs=[attr])

## Well, blow me down!
# tiledb.Array.create(array_url, schema)





gcs://fc-6f27926c-489e-469f-adc5-bb2d9cfe7eec/tiledb/dscatalog


TileDBError: [TileDB::StorageManager] Error: Cannot create array; Array 'gcs://fc-6f27926c-489e-469f-adc5-bb2d9cfe7eec/tiledb/dscatalog' already exists